In [1]:
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from architectures import *
from collections import Counter
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from torchvision import transforms, datasets
from utils import *

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 37
ROOT_PATH = os.getcwd()
os.makedirs(DATASET_PATH := os.path.join(ROOT_PATH, "dataset"), exist_ok=True)
os.makedirs(MODELS_PATH := os.path.join(ROOT_PATH, "models"), exist_ok=True)

BATCH_SIZE = 32
TRAIN_RATIO, DEV_RATIO, TEST_RATIO = 0.8, 0.1, 0.1
LBP_ENGINEERING = False

LEARNING_RATE = 0.001
NUM_EPOCHS = 50
PATIENCE = 10

IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224

seed_functions(SEED)

c:\Users\gregory\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading

In [2]:
T = transforms.Compose([transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)), transforms.ToTensor()])
dataset = datasets.ImageFolder(DATASET_PATH, transform=T)

def split_data(dataset):
	labels = [tp[1] for tp in dataset]
	print("Class balance:", dict(Counter(labels)))
	train_set, temp_set = train_test_split(dataset, test_size=(1 - TRAIN_RATIO), random_state=SEED, stratify=labels)
	temp_labels = [tp[1] for tp in temp_set]
	test_set, dev_set = train_test_split(temp_set, test_size=(TEST_RATIO / (DEV_RATIO + TEST_RATIO)), random_state=SEED, stratify=temp_labels)
	if LBP_ENGINEERING:
		return LBPDataset(train_set), LBPDataset(dev_set), LBPDataset(test_set)
	return train_set, dev_set, test_set

train_set, dev_set, test_set = split_data(dataset)
train_generator = torch.Generator().manual_seed(SEED)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, generator=train_generator)
dev_loader = torch.utils.data.DataLoader(dev_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

Class balance: {0: 1559, 1: 1559}


Training

In [3]:
model = CNV2B().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

def freeze_model(model, frozen_dict):
	if not getattr(model, "freezable", False):
		return
	cfg = frozen_dict.get(model.id)
	if cfg is None:
		return
	prefix = cfg["freeze_until"]
	print(f"[Freezing] model.id={model.id}, freeze_until={prefix}")
	for name, param in model.named_parameters():
		print(f"  - {name}")
		param.requires_grad = False
		if name.startswith(prefix):
			break

def train_model(model, train_loader, dev_loader, criterion, optimizer):
	# Create model folder
	model_folder = os.path.join(MODELS_PATH, f"{model.id}")
	os.makedirs(model_folder, exist_ok=True)

	best_dev_loss = float("inf")
	best_model_path = os.path.join(model_folder, "best.pth")
	patience_counter = 0
	train_losses = []
	dev_losses = []

	for epoch in range(NUM_EPOCHS):
		# Train model on train set
		model.train()
		total_loss = 0
		for images, labels in train_loader:
			images, labels = images.to(DEVICE), labels.to(DEVICE)
			optimizer.zero_grad()
			outputs = model(images)
			loss = criterion(outputs, labels)
			loss.backward()
			optimizer.step()
			total_loss += loss.item()
		train_loss = total_loss / len(train_loader)
		train_losses.append(train_loss)

		# Evaluate model on dev set
		model.eval()
		dev_loss = 0
		with torch.no_grad():
			for images, labels in dev_loader:
				images, labels = images.to(DEVICE), labels.to(DEVICE)
				outputs = model(images)
				dev_loss += criterion(outputs, labels).item()
		dev_loss /= len(dev_loader)
		dev_losses.append(dev_loss)

		print(f"Epoch: {epoch+1} of {NUM_EPOCHS} - Training loss: {train_loss:.4f} - Validation loss: {dev_loss:.4f}")

		# Save model epoch checkpoint
		torch.save(model.state_dict(), os.path.join(model_folder, f"epoch={epoch+1}-dev_loss={dev_loss:.4f}.pth"))
		if dev_loss < best_dev_loss:
			best_dev_loss = dev_loss
			torch.save(model.state_dict(), best_model_path)  # Save best model
			patience_counter = 0  # Reset patience counter
		else:
			patience_counter += 1
			# Stop early if needed
			if patience_counter >= PATIENCE:
				print("Stopping early")
				break

	# Save losses
	loss_data = {"train_losses": train_losses, "dev_losses": dev_losses}
	with open(os.path.join(model_folder, "losses.json"), "w") as f:
		json.dump(loss_data, f)

# freeze_model(model, {"RN50": {"freeze_until": "layer3"}})
train_model(model, train_loader, dev_loader, criterion, optimizer)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (229376x7 and 1024x2)

Post-training

In [ ]:
def visualise_loss(model):
	# Load losses
	model_folder = os.path.join(MODELS_PATH, f"{model.id}")
	with open(os.path.join(model_folder, "losses.json"), "r") as f:
		loss_data = json.load(f)
	train_losses = loss_data["train_losses"]
	dev_losses = loss_data["dev_losses"]

	plt.figure(figsize=(8, 5))
	epochs = range(1, len(train_losses) + 1)
	plt.plot(epochs, train_losses, "b-", label="Training loss")
	plt.plot(epochs, dev_losses, "r-", label="Validation loss")

	# Annotate the best loss
	best_epoch = np.argmin(dev_losses) + 1
	best_loss = min(dev_losses)
	plt.plot(best_epoch, best_loss, "ro", markersize=8, label=f"Best loss: {best_loss:.4f}")

	plt.xlabel("Epoch")
	plt.ylabel("Loss")
	plt.legend()
	plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(1))
	plt.tight_layout()
	plt.show()

def evaluate_model(model, dataloader):
	model.eval()
	Y_hat, Y = [], []
	with torch.no_grad():
		for x, y in dataloader:
			logits = model(x.to(DEVICE))
			Y_hat.append(torch.argmax(logits, dim=1).cpu())
			Y.append(y.view(-1))
	Y_hat, Y = torch.cat(Y_hat), torch.cat(Y)
	tn, fp, fn, tp = confusion_matrix(Y, Y_hat).ravel()
	binary_f1 = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) != 0 else 0
	macro_f1 = f1_score(Y, Y_hat, average="macro")
	return {
		"tp": int(tp),
		"tn": int(tn),
		"fp": int(fp),
		"fn": int(fn),
		"macro_f1": float(macro_f1)
	}

m = model.__class__().to(DEVICE)
visualise_loss(m)

m.load_state_dict(torch.load(os.path.join(MODELS_PATH, f"{m.id}", "best.pth")))
print(evaluate_model(m, test_loader))